In [256]:
import pandas as pd
import numpy as np

In [281]:
INPUT_CSV_FILE_NAME = 'extracted_movies.csv'
MOVIES_DETAILS_CSV_NAME = 'movies_details.csv'
MOVIES_CAST_CSV_NAME = 'movies_cast.csv'
MOVIES_REVIEWS_CSV_NAME = 'movies_reviews.csv'
END_REVIEW_TOKEN = '$&&&&$'

In [287]:
movies = pd.read_csv(INPUT_CSV_FILE_NAME)
movies_details = movies[['Title', 'Year', 'Director', 'Movie summary']].copy()

In [288]:
movies_details.loc[:,'Title'] = movies_details.loc[:,'Title'].str.strip()
# oczyszczanie kolumny 'Title' z niepotrzebnych białych znaków

In [290]:
def from_string_to_dict_on_cast(cast_string):
    #przetworzenie stringu na listę aktorzy i role w jednej tabeli
    #aktorzy nieparzyści, role parzyste
    cast_string = list(cast_string.replace("\n", "")
                              .strip()
                              .replace("\\n", "")
                              .replace("'", "").replace("[  ", "")
                              .replace("]", "")
                              .replace("               ...          ", ",   ")
                              .split(",   "))
    #przekonwertowanie listy na słownik aktor:rola
    cast_dict = dict(zip(cast_string[::2], cast_string[1::2]))
    return cast_dict

In [291]:
movies_details.loc[:,'Year'] = movies_details.loc[:,'Year'].str.extract('(\d+)').astype(int)
# wyciągnięcie numeru z kolumny 'Year' i zmiana na int

In [292]:
movies_details.loc[:,'Director'] = movies_details.loc[:,'Director'].str.replace("\nDirector:\n", "").str.replace("\nDirectors:\n", "")
# wyciągnięcie z kolumny 'Title' imię i nazwisko reżysera

In [293]:
movies_details

,Title,Year,Director,Movie summary
0,The Irishman,2019,Martin Scorsese,A mob hitman recalls his possible involvement ...
1,Frozen II,2019,"Chris Buck, Jennifer Lee","Anna, Elsa, Kristoff, Olaf and Sven leave Aren..."
2,Knives Out,2019,Rian Johnson,A detective investigates the death of a patria...


In [294]:
movies_details.to_csv(MOVIES_DETAILS_CSV_NAME, index=False)

In [ ]:
#####################################################################################

In [307]:
movies_reviews = movies.filter(['Title', 'Reviews'], axis=1)

In [309]:
def format_reviews(reviews_string):
    '''Formating reviews string and separating them to list elements.'''
    END_REVIEW_TOKEN = '$&&&&$'
    reviews_list = list(reviews_string.replace("[", "")
                                      .replace("]", "")
                                      .replace("\'", "")
                                      .replace("\\", "")
                                      .replace("\\t", "")
                                      .replace("\\s", "")
                                      .replace("\\m", "")
                                      .replace(', "', " ")
                                      .strip()
                                      .replace(END_REVIEW_TOKEN + '"', END_REVIEW_TOKEN)
                                      .split(END_REVIEW_TOKEN))
     
    del reviews_list[-1]
    return reviews_list

In [310]:
movies_reviews.loc[:,'Reviews'] = movies_reviews.loc[:,'Reviews'].apply(format_reviews)

AttributeError: 'list' object has no attribute 'replace'

In [311]:
def create_row_for_each_list_element(movies, list_column_to_split):
    '''Creating row in dataframe for each list element.'''

    extended_reviews_df = pd.DataFrame({
                        col:np.repeat(movies[col].values, movies[list_column_to_split].str.len())
                        for col in movies.columns.drop(list_column_to_split)
                        }).assign(**{reviews_col:np.concatenate(movies[list_column_to_split].values)})[movies.columns]
    return extended_reviews_df

In [314]:

movies_reviews = create_row_for_each_list_element(movies_reviews, 'Reviews')

In [316]:
movies_reviews.to_csv(MOVIES_REVIEWS_CSV_NAME, index=False)

In [ ]:
###########################################################################################

In [ ]:
def from_string_to_dict_on_cast(cast_string):
    '''Modifing cast string and saving results to dictionary'''
    #przetworzenie stringu na listę aktorzy i role w jednej tabeli
    #aktorzy nieparzyści, role parzyste
    cast_string = list(cast_string.replace("\n", "")
                              .strip()
                              .replace("\\n", "")
                              .replace("'", "").replace("[  ", "")
                              .replace("]", "")
                              .replace("               ...          ", ",   ")
                              .split(",   "))
    #przekonwertowanie listy na słownik aktor:rola
    cast_dict = dict(zip(cast_string[::2], cast_string[1::2]))
    return cast_dict
movies.loc[:,'Cast'] = movies.loc[:,'Cast'].apply(from_string_to_dict_on_cast)